
# Convolutional Neural Network(CNN)
In Tensorflow, a Convolutional Neural Network (CNN) is a type of deep learning model that is 
commonly used for image classification,object detection, and other computer vision tasks.
A CNN consists of mutliply layers, each of which performs a different operation on the input data.
The first layer in CNN is typically a convolutional layer, which applies  a set of learnable filters to the input
impage to extract features.
The output of the convolutioal layer in then passed through one or more pooling layers, which reduce the spatial dimensions
of the feature maps while preserving important features.
After several convolutional and pooling layers, the output is typically flatteend and passed through one or more fully connected layers,
which perform classification or regression on the extract features.
The final output layer of a CNN typically uses a softmax activation function to produce a probability distribution over the possible
classes.
In tensorflow, you an create a CNN using a high-level 
keras API or by directly defining the layers and operations using the tensorflow API

# Trying the Predicting the Shape of the Image

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Saving the Path of Image folder to a variable

In [2]:
image_dir=Path('/kaggle/input/basicshapes/shapes/shapes')

# Saving the path of each image to a list

In [3]:
filepath=list(image_dir.glob(r'**/*.png'))
filepath

[PosixPath('/kaggle/input/basicshapes/shapes/shapes/circles/drawing(16).png'),
 PosixPath('/kaggle/input/basicshapes/shapes/shapes/circles/drawing(9).png'),
 PosixPath('/kaggle/input/basicshapes/shapes/shapes/circles/drawing(95).png'),
 PosixPath('/kaggle/input/basicshapes/shapes/shapes/circles/drawing(49).png'),
 PosixPath('/kaggle/input/basicshapes/shapes/shapes/circles/drawing(90).png'),
 PosixPath('/kaggle/input/basicshapes/shapes/shapes/circles/drawing(21).png'),
 PosixPath('/kaggle/input/basicshapes/shapes/shapes/circles/drawing(68).png'),
 PosixPath('/kaggle/input/basicshapes/shapes/shapes/circles/drawing(86).png'),
 PosixPath('/kaggle/input/basicshapes/shapes/shapes/circles/drawing(63).png'),
 PosixPath('/kaggle/input/basicshapes/shapes/shapes/circles/drawing(14).png'),
 PosixPath('/kaggle/input/basicshapes/shapes/shapes/circles/drawing(80).png'),
 PosixPath('/kaggle/input/basicshapes/shapes/shapes/circles/drawing(52).png'),
 PosixPath('/kaggle/input/basicshapes/shapes/shapes/c

In [4]:
os.path.split(os.path.split('/kaggle/input/basicshapes/shapes/circles/drawing(16).png')[0])[1]

'circles'

In [5]:
labels=list(map(lambda x:os.path.split(os.path.split(x)[0])[1],filepath))

In [6]:
filepath=pd.Series(filepath,name='Filepath').astype(str)
label=pd.Series(labels,name='Labels')
image_df=pd.concat([filepath,label],axis=1)
image_df

,Filepath,Labels
0,/kaggle/input/basicshapes/shapes/shapes/circle...,circles
1,/kaggle/input/basicshapes/shapes/shapes/circle...,circles
2,/kaggle/input/basicshapes/shapes/shapes/circle...,circles
3,/kaggle/input/basicshapes/shapes/shapes/circle...,circles
4,/kaggle/input/basicshapes/shapes/shapes/circle...,circles
...,...,...
295,/kaggle/input/basicshapes/shapes/shapes/square...,squares
296,/kaggle/input/basicshapes/shapes/shapes/square...,squares
297,/kaggle/input/basicshapes/shapes/shapes/square...,squares
298,/kaggle/input/basicshapes/shapes/shapes/square...,squares


# Spitting the DataFrame into train and test

In [7]:
train_df,test_df=train_test_split(image_df,train_size=0.7,shuffle=True,random_state=1)

# Loading the Image Data

In [8]:
train_generator=tf.keras.preprocessing.image.ImageDataGenerator(
rescale=1./255,
validation_split=0.2)

test_generator=tf.keras.preprocessing.image.ImageDataGenerator(
rescale=1./255)

In [20]:
train_images=train_generator.flow_from_dataframe(
dataframe=train_df,
x_col='Filepath',
y_col='Labels',
target_size=(28,28),
color_model='grayscale',
class_model='categorical',
batch_size=32,
shuffle=True,
seed=42,
subset='training')

val_images=train_generator.flow_from_dataframe(
dataframe=train_df,
x_col='Filepath',
y_col='Labels',
target_size=(28,28),
color_model='grayscale',
class_model='categorical',
batch_size=32,
shuffle=True,
seed=42,
subset='validation')


test_images=test_generator.flow_from_dataframe(
dataframe=test_df,
x_col='Filepath',
y_col='Labels',
target_size=(28,28),
color_model='grayscale',
class_model='categorical',
batch_size=32,
shuffle=False
)

Found 168 validated image filenames belonging to 3 classes.
Found 42 validated image filenames belonging to 3 classes.
Found 90 validated image filenames belonging to 3 classes.


In [10]:
test_images

# Training the Model

In [21]:
inputs=tf.keras.Input(shape=(28,28,1))
x=tf.keras.layers.Conv2D(filters=6,kernel_size=(3,3),activation='relu')(inputs)
x=tf.keras.layers.MaxPooling2D()(x)

x=tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation='relu')(inputs)
x=tf.keras.layers.MaxPooling2D()(x)


x=tf.keras.layers.GlobalAveragePooling2D()(x)
x=tf.keras.layers.Dense(64,activation='relu')(x)
x=tf.keras.layers.Dense(64,activation='relu')(x)
outputs=tf.keras.layers.Dense(3,activation='softmax')(x)
model=tf.keras.Model(inputs=inputs,outputs=outputs)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [22]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 13, 13, 16)       0         
 2D)                                                             
                                                                 
 global_average_pooling2d_2   (None, 16)               0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 64)                1088      
                                                                 
 dense_7 (Dense)             (None, 64)                4160

In [26]:
history=model.fit(train_images,
                 validation_data=val_images,
                 epochs=100,
                 callbacks=[
                 tf.keras.callbacks.EarlyStopping(
                 monitor='val_loss',
                 patience=3,
                restore_best_weights=True
                 
                 )
                           ]                 )

Epoch 1/100


NotFoundError: Graph execution error:

Detected at node 'model_2/conv2d_5/Relu' defined at (most recent call last):
    File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 1041, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 541, in run_forever
      self._run_once()
    File "/opt/conda/lib/python3.7/asyncio/base_events.py"

In [ ]:
input

In [ ]:
x

x

In [ ]:
x